# Report merger
Work in progress... A tool to quickly fetch some numbers in annual reports.

In [4]:
import pandas as pd
import pdfplumber
from glob import glob
import re
from math import nan
from tabula import read_pdf
from IPython.display import display

# 1. Get file list

In [6]:
file = "/Users/paul/Notebooks/donnees-clubs/1819_comptes_individuels_clubs_DIGITAL_FR.pdf"

# 2. Find the right page

In [19]:
re.search('Table des mati[èe]res', 'Bonjour\n\r Table des matieres')

<_sre.SRE_Match object; span=(10, 28), match='Table des matieres'>

In [15]:
# 1 - loop through pdf
with pdfplumber.open(file) as pdf:
    found = False
    currentPage = 0
    pdfLength = len(pdf.pages)
    if row['Page no'] == row['Page no'] and 1 == 2: # skip condition
        print('Page number already set for', row['File path'])
        currentPage = int(row['Page no'])
        print(currentPage)
    else:
        print('“{}” has {} pages'.format(row['File path'], pdfLength))

    # 2 - loop through pages
    while found == False and currentPage < pdfLength:
        pageSelection = pdf.pages[currentPage]

        try:
            pageText = pageSelection.extract_text()
        except ValueError as e:
            print('Error when opening Pdf page for {}: {}'.format(outputfile, e))
            pageText = False

        if pageText:
            # Skip Table of Contents...
            if re.search('Table des mati[èe]res', pageText, flags=re.IGNORECASE):
                print('Table of Contents -> we skip')
            else:
                # we could also get “Bilans{0,1} consolidés{0,1}”
                result = re.search('Comptes{0,1} de résultat|Comptes{0,1} de pertes|Comptes{0,1} de profits et pertes', pageText, flags=re.IGNORECASE)
                if result:
                    print('Match found: “{}”'.format(result.group()))
                    found = True
                    df.at[i, 'Page no'] = currentPage
                else:
                    print('No match found on page', currentPage)
        currentPage += 1

    if not result:
        print('NO RESULT for {}. Excerpt:'.format(row['File path']), pageText.split('\n')[0])

    print() # empty line

“pdf/2007.pdf” has 1 pages
Match found: “Comptes de profits et pertes”

“pdf/2007_rf.pdf” has 25 pages
No match found on page 0
Table of Contents -> we skip
No match found on page 2
Match found: “Comptes de profits et pertes”

“pdf/2008.pdf” has 1 pages
Match found: “Comptes de profits et pertes”

“pdf/2009.pdf” has 1 pages
Match found: “Compte de profits et pertes”

“pdf/2010_RF_GeneveAeroport.pdf” has 27 pages
No match found on page 0
No match found on page 1
Match found: “Comptes de profits et pertes”

“pdf/2011.pdf” has 1 pages
Match found: “Compte de profits et pertes”

“pdf/2012.pdf” has 1 pages
Match found: “Compte de profits et pertes”

“pdf/2014.pdf” has 1 pages
Match found: “Compte de profits et pertes”

“pdf/2015.pdf” has 1 pages
Match found: “Compte de résultat”

“pdf/2016.pdf” has 1 pages
Match found: “Compte de résultat”

“pdf/2017.pdf” has 1 pages
Match found: “Compte de résultat”

“pdf/Resultats_2017.pdf” has 1 pages
Match found: “Compte de résultat”



# 3. Fetch tables

In [7]:
# list of dataframes
d = []

for i, row in df.iterrows():
    data = read_pdf(row['File path'], pages=int(row['Page no']))
    print('Getting numbers in {} page {}…'.format(row['File path'], int(row['Page no'])), end=' ')
    if isinstance(data, pd.core.frame.DataFrame):
        d.append({'year': row['Year'], 'data': data.fillna('')})
        print('DONE')
    else:
        print('\nWARNING Tabula could not extract data in {} page {}'.format(row['File path'], int(row['Page no'])))

Getting numbers in pdf/2007.pdf page 0… DONE
Getting numbers in pdf/2007_rf.pdf page 3… DONE
Getting numbers in pdf/2008.pdf page 0… DONE
Getting numbers in pdf/2009.pdf page 0… DONE
Getting numbers in pdf/2010_RF_GeneveAeroport.pdf page 2… 
WARNING Tabula could not extract data in pdf/2010_RF_GeneveAeroport.pdf page 2
Getting numbers in pdf/2011.pdf page 0… DONE
Getting numbers in pdf/2012.pdf page 0… DONE
Getting numbers in pdf/2014.pdf page 0… DONE
Getting numbers in pdf/2015.pdf page 0… DONE
Getting numbers in pdf/2016.pdf page 0… DONE
Getting numbers in pdf/2017.pdf page 0… DONE
Getting numbers in pdf/Resultats_2017.pdf page 0… DONE


# 4. Some data cleaning here

In [8]:
# TODO
# - transpose
# - convert to same unit (thousands, millions etc.)
# - clean headers
# - skip empty lines (beware of multiline / merged cells)
# ...

In [84]:
t_d = [] # transformed dfs
cols = ['Label', 'Note', 'Report year', 'Previous year']

# Preview and transform
for _d in d:
    print(_d['year'], '> lines:', len(_d['data']), '/ shape:', _d['data'].shape)
    processed_table = _d['data'].copy()
    
    if len(processed_table.columns) < 4:
        processed_table.insert(1, 'Note', '') # to preserve column order

    # Uncomment for preview
    #display(processed_table)

    # Harmonize column names
    
    if _t_d['year'] in processed_table.columns:
        print('Year already in columns')
    else:
        print('No year in headers')
        # TODO: change header to first row
        firstRow = pd.DataFrame([dd.columns], columns=cols).fillna('')

        processed_table = pd.concat([firstRow, dd])
        
    processed_table.columns = cols
    processed_table['File year'] = _d['year']

    t_d.append(processed_table)


2007 > lines: 40 / shape: (40, 4)
No year in headers
2007 > lines: 26 / shape: (26, 4)
No year in headers
2008 > lines: 14 / shape: (14, 4)
No year in headers
2009 > lines: 43 / shape: (43, 4)
No year in headers
2011 > lines: 35 / shape: (35, 4)
No year in headers
2012 > lines: 41 / shape: (41, 4)
No year in headers
2014 > lines: 44 / shape: (44, 4)
No year in headers
2015 > lines: 45 / shape: (45, 4)
No year in headers
2016 > lines: 47 / shape: (47, 4)
No year in headers
2017 > lines: 40 / shape: (40, 4)
No year in headers
2017 > lines: 40 / shape: (40, 4)
No year in headers


# 5. Quick export to single 4-column df

In [87]:
df_all = pd.concat(t_d)

In [88]:
df_all

,Label,Note,Report year,Previous year,File year
0,Label,Note,Report year,Previous year,2007
0,,KCHF,KCHF,,2007
1,PRODUITS,,,,2007
2,Redevances d’atterrissages,44’444,43’066,,2007
3,Redevances passagers,201’238,191’679,,2007
4,Autres redevances aéronautiques,14’691,14’111,,2007
5,Total des redevances aéroportuaires,260’373,248’856,,2007
6,Redevances commerciales,96’728,92’735,,2007
7,Parkings,43’669,42’542,,2007
8,Loyers et droits de superficie,26’774,21’944,,2007


In [89]:
df_all.to_excel('df_all.xls')